# Scraping the Gas Price table
### Price is updated daily

In [1]:
#dependencies
import pandas as pd
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import requests

# Get Abbreviations by state

In [2]:
#Decided to find a website to scrape the state abbreviations
url_abv = "https://abbreviations.yourdictionary.com/articles/state-abbrev.html"

In [3]:
response = requests.get(url_abv)
soup = BeautifulSoup(response.text, 'html.parser')

In [4]:
#HTML inspect found the values in the li tag
results = soup.find_all('li')

In [5]:
# total length of li
len(results)

199

In [6]:
#strip out only the first 50 li tags that contain the abbreviations
ABV = []

In [7]:
for i in range(0, 50):
    results[i].text
    ABV.append(results[i].text)

In [8]:
len(ABV)

50

In [9]:
#Take the new list and seperate the abbreviations into a new list
ABV_new = []
State_name = []

In [10]:
for i in range(0, 50):
    ABV_clean = ABV[i].split(' - ')[1]
    ABV_new.append(ABV_clean.replace(' ',''))

In [11]:
for i in range(0, 50):
    ABV_clean = ABV[i].split(' - ')[0]
    State_name.append(ABV_clean.replace(' ',''))

In [15]:
All_States_df = pd.DataFrame({'State':State_name,
                           'Abbreviation':ABV_new                   
})

In [16]:
All_States_df

,State,Abbreviation
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA
5,Colorado,CO
6,Connecticut,CT
7,Delaware,DE
8,Florida,FL
9,Georgia,GA


# Getting the Gas Prices per State

In [17]:
#Appened the abbriviations list to the URL to get all 50 states url
url = "https://gasprices.aaa.com/?state="

GasPrice_State_urls = []

for i in range(0, 50):
    GasPrice_State_urls.append(url+ABV_new[i])

In [18]:
Scraped_df = []
Final_df = pd.DataFrame()

In [19]:
GasPrice_State_urls

['https://gasprices.aaa.com/?state=AL',
 'https://gasprices.aaa.com/?state=AK',
 'https://gasprices.aaa.com/?state=AZ',
 'https://gasprices.aaa.com/?state=AR',
 'https://gasprices.aaa.com/?state=CA',
 'https://gasprices.aaa.com/?state=CO',
 'https://gasprices.aaa.com/?state=CT',
 'https://gasprices.aaa.com/?state=DE',
 'https://gasprices.aaa.com/?state=FL',
 'https://gasprices.aaa.com/?state=GA',
 'https://gasprices.aaa.com/?state=HI',
 'https://gasprices.aaa.com/?state=ID',
 'https://gasprices.aaa.com/?state=IL',
 'https://gasprices.aaa.com/?state=IN',
 'https://gasprices.aaa.com/?state=IA',
 'https://gasprices.aaa.com/?state=KS',
 'https://gasprices.aaa.com/?state=KY',
 'https://gasprices.aaa.com/?state=LA',
 'https://gasprices.aaa.com/?state=ME',
 'https://gasprices.aaa.com/?state=MD',
 'https://gasprices.aaa.com/?state=MA',
 'https://gasprices.aaa.com/?state=MI',
 'https://gasprices.aaa.com/?state=MN',
 'https://gasprices.aaa.com/?state=MS',
 'https://gasprices.aaa.com/?state=MO',


In [ ]:
for i in range(0, 50):
    agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36\
    (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'
    
    url = GasPrice_State_urls[i]
    
    request = Request(url, headers={'User-Agent': agent})

    html = urlopen(request).read().decode()
    
    tables = pd.read_html(html)
    
    df = tables[0]
    
    df["state"] = ABV_new[i]
    
    Scraped_df.append(df)

In [ ]:
for i in range(0,49):
    Final_df = Final_df.append(Scraped_df[i])

In [ ]:
Final_df = Final_df.rename(columns={'Unnamed: 0': 'Averages',
                                    'Mid-Grade':'MidGrade'
                                   })

In [ ]:
Final_df.set_index('state')

# SQL connection

In [20]:
import pymysql
pymysql.install_as_MySQLdb()

from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

from sqlalchemy import Column, Integer, String, Float

from sqlalchemy import create_engine

In [21]:
dbuser = 'root'
dbpassword = 'Rosenhyf1!'
dbhost = 'localhost'
dbport = '3306'
dbname= 'etl_db'

engine = create_engine(f"mysql://{dbuser}:{dbpassword}@{dbhost}:{dbport}/{dbname}")
Base.metadata.create_all(engine)

In [22]:
from sqlalchemy.orm import Session
session = Session(bind=engine)

In [ ]:
Final_df.to_sql('gasprices', engine, index=True, if_exists='replace')

In [25]:
All_States_df.to_sql('states', engine, index=True, if_exists='replace')